<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Language Models

**Note: For other Non-OpenAI models, you can check out: https://python.langchain.com/docs/modules/model_io/models/llms/ although the interface is extremely similar, its just that the results from .generation calls will have differentinformation depending on the service you use.**

## Text Model Connection

In [17]:
from langchain.llms import OpenAI
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os

You can stored your API key however you prefer, its common to set it as an environment variable, for example:

In [18]:
#os.environ['OpenAI_API_KEY'] = 'ce53c5fce80c4503927244333e40634c'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://wdc-chat-llm.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "ce53c5fce80c4503927244333e40634c"
os.environ["OPENAI_API_VERSION"] = "2024-06-01"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"
SecretStr = os.environ["AZURE_OPENAI_API_KEY"]

In [3]:
print(os.environ['OPenAI_API_KEY'])
print(os.environ["AZURE_OPENAI_API_KEY"])

ce53c5fce80c4503927244333e40634c
ce53c5fce80c4503927244333e40634c


Note, that LangChain automatically looks up for any environment variable with the name OPENAI_API_KEY automatically when making a connection to OpenAI. Alternatively, you could just pass in the openai key via a string (not very secure, but okay for your own local projects), or even just save it somewhere on your computer in a text file and then read it in, for example:

In [5]:
f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
api_key = f.read()

In [19]:
#llm = OpenAI(openai_api_key=api_key)
#llm = AzureOpenAI(model="gpt -3.5-turbo-instruct")
llm = AzureChatOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version = os.environ["OPENAI_API_VERSION"],
    temperature= 0.9
)
parser = StrOutputParser()

## Text Model Call

This is the simplest way to get a text autocomplete:

In [20]:
result = llm.invoke('Here is a fun fact about Pluto:')
parser.invoke(result)

'Pluto, once considered the ninth planet of our solar system, was reclassified as a "dwarf planet" by the International Astronomical Union (IAU) in 2006. It is located in the Kuiper Belt, a region of the solar system beyond Neptune that is filled with small icy bodies. Despite its reclassification, many still hold a sentimental attachment to Pluto as the "ninth planet." One fascinating fact about Pluto is that it has five known moons: Charon, which is the largest and nearly half the size of Pluto itself, and four smaller moons named Nix, Hydra, Kerberos, and Styx.'

You can also use generate to get full output with more info:

In [ ]:
# NEEDS TO BE A LIST, EVEN FOR JUST ONE STRING
result = llm.generate(['Here is a fun fact about Pluto:',
                       'Here is a fun fact about Mars:']
                     )

c:\Program Files\Python311\Lib\site-packages\langchain_openai\chat_models\base.py:369: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [22]:
result.model_json_schema()

{'$defs': {'BaseMessage': {'additionalProperties': True,
   'description': 'Base abstract message class.\n\nMessages are the inputs and outputs of ChatModels.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type', 'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Id'}},
   'required': ['content', 'type'],
   'title': 'BaseMessage',
   'type': 'object'},
  'BaseMessageChunk': {'additionalProperties': True,
   'description': 'Message chunk, which can be concatenated with other Message chunks.',
   'properties': {'co

In [23]:
result.llm_output

{'token_usage': {'completion_tokens': 178,
  'prompt_tokens': 30,
  'total_tokens': 208,
  'completion_tokens_details': None,
  'prompt_tokens_details': None},
 'model_name': None,
 'system_fingerprint': 'fp_67802d9a6d'}

# Chat Models

The most popular models are actually chat models, that have a System Message and then a series of Assistant and Human Messages

In [11]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(openai_api_key=api_key)

In [12]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [13]:
result = chat([HumanMessage(content="Can you tell me a fact about Earth?")])

In [14]:
result

AIMessage(content='One fact about Earth is that it is the only known planet to support life.', additional_kwargs={}, example=False)

In [15]:
result.content

'One fact about Earth is that it is the only known planet to support life.'

In [16]:
result = chat([SystemMessage(content='You are a very rude teenager who only wants to party and not answer questions'),
               HumanMessage(content='Can you tell me a fact about Earth?')])

In [17]:
result.content

"Ugh, seriously? Fine, here's a fact for you. Earth is the third planet from the Sun in our solar system. Now, can we move on to something more exciting? Like, I don't know, partying or something?"

In [18]:
# NEEDS TO BE A LIST!
result = chat.generate(
                [
                [SystemMessage(content='You are a University Professor'),
               HumanMessage(content='Can you tell me a fact about Earth?')]
                ]
                    )

In [19]:
result

LLMResult(generations=[[ChatGeneration(text='Certainly! One interesting fact about Earth is that it is the only known planet in our solar system where water exists in all three states: solid (ice), liquid (water), and gas (water vapor). This unique combination of water in different states allows for the existence of life as we know it on Earth.', generation_info=None, message=AIMessage(content='Certainly! One interesting fact about Earth is that it is the only known planet in our solar system where water exists in all three states: solid (ice), liquid (water), and gas (water vapor). This unique combination of water in different states allows for the existence of life as we know it on Earth.', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 25, 'completion_tokens': 62, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('075174db-b350-4076-8b34-f43ea59169dd'))])

In [20]:
result.llm_output

{'token_usage': {'prompt_tokens': 25,
  'completion_tokens': 62,
  'total_tokens': 87},
 'model_name': 'gpt-3.5-turbo'}

In [21]:
result.generations[0][0].text

'Certainly! One interesting fact about Earth is that it is the only known planet in our solar system where water exists in all three states: solid (ice), liquid (water), and gas (water vapor). This unique combination of water in different states allows for the existence of life as we know it on Earth.'

## Extra Parameters and Args

Here we add in some extra parameters and args, note we chose some pretty extreme values!

In [22]:
result = chat([HumanMessage(content='Can you tell me a fact about Earth?')],
                 temperature=2,presence_penalty=1,max_tokens=100)

In [25]:
result.content

'Cycling remains one of the mosPopular physical activities performed on Earth'

# Caching

Making the same exact request often? You could use a cache to store results **note, you should only do this if the prompt is the exact same and the historical replies are okay to return**.

In [26]:
import langchain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=api_key)

In [27]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a fact about Mars")

'One interesting fact about Mars is that it has the largest volcano in the solar system. Named Olympus Mons, this shield volcano stands about 13.6 miles (22 kilometers) high and spans approximately 370 miles (600 kilometers) in diameter. It is nearly three times the height of Mount Everest, making it the tallest volcano and one of the largest known volcanoes in the entire solar system.'

In [28]:
# You will notice this reply is instant!
llm.predict('Tell me a fact about Mars')

'One interesting fact about Mars is that it has the largest volcano in the solar system. Named Olympus Mons, this shield volcano stands about 13.6 miles (22 kilometers) high and spans approximately 370 miles (600 kilometers) in diameter. It is nearly three times the height of Mount Everest, making it the tallest volcano and one of the largest known volcanoes in the entire solar system.'

You can also use SQLite Caches: https://python.langchain.com/docs/modules/model_io/models/chat/how_to/chat_model_caching#sqlite-cache